In [76]:
import pandas as pd

In [77]:
df = pd.read_csv('SLR_coastel.csv')

In [78]:
coastalDf = pd.read_csv('Mod_coastalEcon.csv')
oceanDf = pd.read_csv('oceanEconomy_ENOW.csv')

In [79]:
df = df.drop(columns = ['time', 'latitude', 'longitude'])

In [80]:
df = df.groupby(['State', 'year']).agg({'adt': 'mean',
                                               'sla': 'mean',
                                               'ugos': 'mean',
                                               'ugosa': 'mean',
                                               'vgos': 'mean',
                                               'vgosa': 'mean'}).reset_index()

# Print the resulting DataFrame
print(df)

    State  year       adt       sla      ugos     ugosa      vgos     vgosa
0      AL  1993  0.102025 -0.086775  0.014312  0.009437 -0.037937 -0.010637
1      AL  1994  0.133313 -0.055488  0.009912  0.005013 -0.005913  0.021413
2      AL  1995  0.126388 -0.062413 -0.037675 -0.042575 -0.013950  0.013388
3      AL  1996  0.163038 -0.025763 -0.019700 -0.024587 -0.030638 -0.003325
4      AL  1997  0.051338 -0.137462 -0.017950 -0.022825 -0.038900 -0.011562
..    ...   ...       ...       ...       ...       ...       ...       ...
475    VA  2018 -0.019975  0.103167 -0.030512 -0.023067 -0.133013 -0.099175
476    VA  2019 -0.068383  0.054758 -0.002750  0.004700 -0.002233  0.031596
477    VA  2020 -0.075617  0.047513 -0.085346 -0.077900 -0.116329 -0.082488
478    VA  2021 -0.004425  0.118712 -0.014721 -0.007292 -0.043079 -0.009254
479    VA  2022 -0.024983  0.098162  0.028667  0.036112  0.015983  0.049817

[480 rows x 8 columns]


In [81]:
oceanDf['sector'].unique()

array(['Ship and Boat Building', 'Tourism and Recreation',
       'Marine Transportation', 'Ocean Economy', 'Marine Construction',
       'Living Resources', 'Offshore Mineral Resources'], dtype=object)

In [82]:
coastalDf['sector'].unique()

array(['Financial Activities', 'Education and Health Services',
       'Information', 'Leisure and Hospitality', 'All Industries Total',
       'Construction', 'Other Services',
       'Professional and Business Services', 'Public Administration',
       'Manufacturing', 'Trade Transportation and Utilities',
       'Natural Resources and Mining'], dtype=object)

In [83]:
print(oceanDf.columns)
print(coastalDf.columns)

Index(['geoid', 'geoName', 'year', 'sector', 'establishments', 'employment',
       'wages', 'gdp'],
      dtype='object')
Index(['geoid', 'geoName', 'year', 'sector', 'establishments', 'employment',
       'wages', 'gdp'],
      dtype='object')


In [84]:
concatenated_df = pd.concat([oceanDf, coastalDf])

In [85]:
df1 = concatenated_df
df2 = df

# Mapping dictionary for all states
state_mapping = {'FL': 'Florida', 'TX': 'Texas', 'LA': 'Louisiana', 'MS': 'Mississippi', 'AL': 'Alabama',
                 'GA': 'Georgia', 'SC': 'South Carolina', 'NC': 'North Carolina', 'VA': 'Virginia',
                 'MD': 'Maryland', 'DE': 'Delaware', 'NJ': 'New Jersey', 'NY': 'New York', 'CT': 'Connecticut',
                 'MA': 'Massachusetts', 'ME': 'Maine'}

# Replace 'State' column values in df2 with their corresponding 'geoName' values
df2['State'] = df2['State'].map(state_mapping)

# Merge the data frames
merged_df = pd.merge(df1, df2, how='inner', left_on=['year', 'geoName'], right_on=['year', 'State'])

print(merged_df)

      geoid   geoName  year                      sector establishments  \
0      1000   Alabama  2017      Ship and Boat Building             56   
1      1000   Alabama  2017      Tourism and Recreation            969   
2      1000   Alabama  2017       Marine Transportation            122   
3      1000   Alabama  2017               Ocean Economy           1370   
4      1000   Alabama  2017         Marine Construction             37   
...     ...       ...   ...                         ...            ...   
2810  51000  Virginia  2013  Offshore Mineral Resources             53   
2811  51000  Virginia  2013      Ship and Boat Building             59   
2812  51000  Virginia  2013      Tourism and Recreation           3253   
2813  51000  Virginia  2013       Marine Transportation            343   
2814  51000  Virginia  2013               Ocean Economy           4046   

     employment       wages             gdp     State       adt       sla  \
0        5148.5   340878916   7680

In [86]:
merged_df['year'].unique()

array([2017, 2018, 2009, 2010, 2011, 2012, 2013, 2020, 2006, 2007, 2008,
       2005, 2016, 2019, 2014, 2015, 2021])

In [87]:
merged_df = merged_df.drop(columns = ['geoid'])

In [88]:
merged_df.dtypes

geoName            object
year                int64
sector             object
establishments     object
employment         object
wages              object
gdp                object
State              object
adt               float64
sla               float64
ugos              float64
ugosa             float64
vgos              float64
vgosa             float64
dtype: object

In [89]:
cols = ['establishments', 'employment', 'wages', 'gdp']

# Convert to numeric, coerce errors to NaN
merged_df[cols] = merged_df[cols].apply(pd.to_numeric, errors='coerce')

# Fill NaN values with 0
#merged_df.fillna(0, inplace=True)
#Drop rows with NaN values in any of the specified columns
merged_df.dropna(subset=cols, inplace=True)

merged_df[cols] = merged_df[cols].astype(float)

In [90]:
merged_df.dtypes

geoName            object
year                int64
sector             object
establishments    float64
employment        float64
wages             float64
gdp               float64
State              object
adt               float64
sla               float64
ugos              float64
ugosa             float64
vgos              float64
vgosa             float64
dtype: object

In [91]:
merged_df.to_csv("SLR_econ.csv")